In [157]:
import pandas as pd
import numpy as np
from pathlib import Path
# import hvplot.pandas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import datetime as dt

%matplotlib inline

In [158]:
filepath = Path("../data/MSFT_EWMA_MACD_signals.csv")

In [159]:
msft_ewma_macd_df = pd.read_csv(filepath)
msft_ewma_macd_df["timestamp"] = pd.to_datetime(msft_ewma_macd_df["timestamp"]).dt.date
msft_ewma_macd_df = msft_ewma_macd_df.set_index("timestamp")
msft_ewma_macd_df.head()

,Short_EWMA,Long_EWMA,Final_Signal,macd,signal_lag,divergence
timestamp,,,,,,
2017-06-01,70.100000,70.100000,0.0,0.000000,0.000000,0.000000
2017-06-02,70.999167,70.961923,0.0,0.037244,0.020691,0.016553
2017-06-05,71.499076,71.435495,0.0,0.063581,0.038269,0.025312
2017-06-06,71.821342,71.738675,0.0,0.082667,0.053309,0.029358
2017-06-07,71.975844,71.889720,0.0,0.086124,0.063070,0.023054


In [160]:
msft_ewma_macd_df.shape

(1260, 6)

In [161]:
X = msft_ewma_macd_df[["Short_EWMA", "Long_EWMA", "macd", "signal_lag"]]
X = X.drop(X.index[1259])
# display(X.head())
# display(X.tail())
X

,Short_EWMA,Long_EWMA,macd,signal_lag
timestamp,,,,
2017-06-01,70.100000,70.100000,0.000000,0.000000
2017-06-02,70.999167,70.961923,0.037244,0.020691
2017-06-05,71.499076,71.435495,0.063581,0.038269
2017-06-06,71.821342,71.738675,0.082667,0.053309
2017-06-07,71.975844,71.889720,0.086124,0.063070
...,...,...,...,...
2022-05-24,261.857360,269.924942,-8.067582,-7.886178
2022-05-25,261.959304,269.376428,-7.417124,-7.792367
2022-05-26,262.565565,269.118915,-6.553349,-7.544564


In [162]:
y = msft_ewma_macd_df["Final_Signal"]
y = y.shift(1).dropna()
y

timestamp
2017-06-02    0.0
2017-06-05    0.0
2017-06-06    0.0
2017-06-07    0.0
2017-06-08    0.0
             ... 
2022-05-25    0.0
2022-05-26    0.0
2022-05-27    0.0
2022-05-31    0.0
2022-06-01    0.0
Name: Final_Signal, Length: 1259, dtype: float64

In [163]:
# from pandas.tseries.offsets import DateOffset

In [164]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [165]:
# training_end = X.index.min() + DateOffset(months= 42) 

# print(training_end)

In [166]:
# X_train = X.loc[training_begin:training_end]
# y_train = y.loc[training_begin:training_end]

In [167]:
# X_test = X.loc[training_end:]
# y_test = y.loc[training_end:]



In [168]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler model to the X training data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test Data using the Scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [169]:

ann = tf.keras.models.Sequential()

number_units= 100

# Layer 1
ann.add(tf.keras.layers.Dense(units=number_units, activation="sigmoid"))

# Layer 2
ann.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))

# Layer 3
ann.add(tf.keras.layers.Dense(units=40, activation="tanh"))

# Output layer
ann.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))   

In [170]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [171]:
# ann.summary()

In [172]:
ann.fit(X_train_scaled, y_train, epochs=500, batch_size=32)

Epoch 1/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6829 - accuracy: 0.5442
Epoch 2/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6626 - accuracy: 0.5700
Epoch 3/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6569 - accuracy: 0.6018
Epoch 4/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6294 - accuracy: 0.6187
Epoch 5/500
32/32 [==============================] - 0s 976us/step - loss: 0.6074 - accuracy: 0.6584
Epoch 6/500
32/32 [==============================] - 0s 1ms/step - loss: 0.5958 - accuracy: 0.6643
Epoch 7/500
32/32 [==============================] - 0s 1ms/step - loss: 0.5952 - accuracy: 0.6375
Epoch 8/500
32/32 [==============================] - 0s 1ms/step - loss: 0.5988 - accuracy: 0.6485
Epoch 9/500
32/32 [==============================] - 0s 1ms/step - loss: 0.5709 - accuracy: 0.6812
Epoch 10/500
32/32 [==============================] - 0s 1ms/step - loss: 0.5628 - accuracy: 0.6912
Epoch 1

In [178]:
model_loss, model_accuracy = ann.evaluate(X_test_scaled, y_test, verbose=2)

8/8 - 0s - loss: 0.0178 - accuracy: 0.9921 - 28ms/epoch - 3ms/step


In [179]:
prediction = (ann.predict(X_test_scaled)> 0.5).astype("int32")

In [180]:
results = pd.DataFrame({"predictions": prediction.ravel(), "actual": y_test})
results.head(10)

,predictions,actual
timestamp,,
2021-04-14,1,1.0
2020-04-13,1,1.0
2021-04-23,1,1.0
2019-08-09,0,0.0
2019-06-17,1,1.0
2017-11-20,0,0.0
2022-03-17,0,0.0
2017-11-02,1,1.0
2020-01-07,1,1.0


In [176]:
from sklearn import svm
from sklearn.metrics import classification_report

In [181]:
report = classification_report(y_test, prediction)


# Display the report
print(report)

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99       143
         1.0       0.98      1.00      0.99       109

    accuracy                           0.99       252
   macro avg       0.99      0.99      0.99       252
weighted avg       0.99      0.99      0.99       252



In [ ]:
# Create the classifier model
svm_model = svm.SVC()
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

array([0., 1., 0., 1., 0., 0., 0., 0., 0., 1.])

In [ ]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display the report
print(training_report)

              precision    recall  f1-score   support

         0.0       0.91      0.87      0.89       553
         1.0       0.85      0.89      0.87       454

    accuracy                           0.88      1007
   macro avg       0.88      0.88      0.88      1007
weighted avg       0.88      0.88      0.88      1007



In [ ]:
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [ ]:
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

              precision    recall  f1-score   support

         0.0       0.95      0.89      0.92       143
         1.0       0.86      0.94      0.90       109

    accuracy                           0.91       252
   macro avg       0.91      0.91      0.91       252
weighted avg       0.91      0.91      0.91       252

